# Simudo tutorial
The goal of this tutorial is to walk through some of the major components of Simudo. We will focus on making a 1D layered structure. 

Simudo is a steady-state coupled Poisson/drift-diffusion solver. It determines the quasi-Fermi-level and current densities of carriers in each band throughout the structure. In order to define a problem in Simudo, we need to define these main pieces
- Regions - possibly overlapping. Regions can have **materials** and **properties**.
- Bands - Semiconductors have 2 (conduction and valence). Intermediate band (IB) materials have 3
- Mesh
- Boundary conditions - Conductive/nonconductive surfaces. Internal BCs at heterojunctions and where IBs end
- Optical fields (if needed)
- Electrooptical processes - Processes that move carriers between bands. Including light absorption and recombination mechanisms such as SRH. 

This tutorial refers to features of Simudo 0.6.4.1.  
Simudo was developed by Eduard Dumitrescu, Matthew Wilkins, and Jacob Krich at the [University of Ottawa](https://krichlab.physics.uottawa.ca). It was first released in 2019 and is available on [github](https://github.com/simudo/simudo).  
It is built on top of the open-source finite-element solver [FEniCS](https://fenicsproject.org/).

In [ ]:
%matplotlib notebook 
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import simudo
print(f"Simudo version: {simudo.__version__} is installed")

### References
[1](https://doi.org/10.1007/s10825-019-01414-3) Eduard C. Dumitrescu, Matthew M. Wilkins, and Jacob J. Krich, 
*Simudo: A device model for intermediate band materials*  
Journal of Computational Electronics **19** 111-127 (2020); arXiv:[1905.11303](https://arxiv.org/abs/1905.11303)

### Obtaining the software
Simudo has been successfully run on Debian and Ubuntu linux distributions as well as using Docker on MacOs and Windows 10 system. Please follow the [installation instructions](https://static.ecd.space/x/simudo-doc/user/install.html)


## Example 1: pn junction

We will begin by setting up a simple problem in Simudo: a pn homojunction. We will consider a p-type semiconductor of thickness $t_p$ and doping $N_A$ connected to an n-type semiconductor of thickness $t_n$ and doping $N_D$. We will have Ohmic (i.e., perfectly conductive) boundary conditions at the boundaries of the system.

A sketch of the structure simulated in this tutorial is shown below.
![pn-sketch.svg](pn-sketch.svg)



This example is modified from `simudo/examples/pn_diode/pn_diode.py`, which has a better structure for the code. Here, we unpack and discuss the various components, in a different order.

### Preamble
Simudo uses the [Pint](https://pint.readthedocs.io/) package to keep track of all units. Since DOLFIN/FEniCS do not know about Pint Quantities, Simudo has a number of utilities to load and unload the units to/from objects. The fundamental object for unit conversions is the `UnitRegistry`. At the beginning of all Simudo evaluations, we establish a unit registry and also set up logging.

In [ ]:
from simudo.util import TypicalLoggingSetup, make_unit_registry
from simudo.fem import setup_dolfin_parameters

U = make_unit_registry(("mesh_unit = 1 micrometer",)) 
TypicalLoggingSetup(filename_prefix="out/output").setup() #filename_prefix should indicate where you want the log files named/stored
setup_dolfin_parameters() 

### Make mesh associated with layers
For 1D problems, it is easiest to use `ConstructionHelperLayeredStructure`, which receives a list of dictionaries, each defining a layer of the device, starting from the left. Each layer should have a `name`, a `material`, a `thickness`, and a `mesh`.  
- `name` is used to identify the layer, for applying rules/parameters/conditions and for setting up output rules to get desired information in the layer.  
- `material` is used to determine material properties and will be described below.  
- `mesh` can be of `type="constant"`, which gives uniform meshing with constant spacing `edge_length` between nodes, or `type="geometric"`, which gives geometrically expanding mesh spacings, growing from the region boundaries. For the geometric mesh, the initial spacing is `start` and the growth factor is `factor`, so it also creates a constant mesh when used with `factor=1`.


In [ ]:
from simudo.mesh import ConstructionHelperLayeredStructure

#Define our parameters for the problem
t_p = 1 #in units of mesh_unit
t_n = 2 #in units of mesh_unit
N_A = 1e18*U("elementary_charge/cm^3")
N_D = 1e17*U("elementary_charge/cm^3")

mesh = dict(type="geometric", start=0.005, factor=1.2)
layers = [
    dict(name='p', material='SimpleMaterial', thickness=t_p, 
         mesh=mesh),
    dict(name='n', material='SimpleMaterial', thickness=t_n,
         mesh=mesh)    
]
ls = ConstructionHelperLayeredStructure()
ls.params=dict(edge_length = 1/20, # default edge length (in units of mesh_unit). Not used with geometric mesh
               simple_overmesh_regions = [], # regions to put extra mesh 
               layers = layers,
               mesh_unit = U.mesh_unit )
ls.run()
mesh_data = ls.mesh_data
#number of mesh points in the x direction:
print(len(ls.interval_1d_tag.coordinates['coordinates']))

Note that we have not yet defined our `SimpleMaterial` class. For now, it is just a label. 

### Define ProblemData
The problem that is to be solved is contained in a `ProblemData` instance. In general, we make three related versions of the the problem, for the three stages of solving
- local charge neutrality
- thermal equilbrium
- full problem

as described in the Reference. The `problemdata` is generally constructed for these three stages using a function, since much of the code is the same for each stage. For this tutorial, we'll make it directly.

## Stage 1 - Local Charge Neutrality

In [ ]:
from simudo.physics import ProblemData

problem_ntrl = ProblemData(goal="local charge neutrality", 
                   mesh_data=mesh_data,
                  unit_registry=U)
pdd = problem_ntrl.pdd

The `pdd` object (it stands for Poisson drift diffusion) is a powerful utility that helps to construct the problem. We will unpack some of its attributes, but there are many.

For this problem, we will create two bands. If no other arguments is given, `pdd.easy_add_band` creates nondegenerate bands that exist in the full simulation domain. Any number of bands can be added, each of which must have a distinct `name`.  
If `name=="CB"`, the band is understood to be a conduction band with negative-charge carriers.  
If `name=="VB"`, the band is understood to be a valence band with positive-charge carriers.  
The `easy_add_band` method can also give any other name to the band, and one can specify the `sign` of the charge carriers directly.

In [ ]:
CB = pdd.easy_add_band(name='CB')
VB = pdd.easy_add_band(name='VB')


### Define spatially varying properties
Simudo solves for electrostatic potential $\phi$, electric field $\vec{E}$, and in each band $k$ the quasi-Fermi level $w_k$ and current density $\vec{j}_k$. In addition to those dynamical quantities, Simudo considers most physical properties of the system to be spatially varying. In this effort, it takes advantage of the powerful symbolic expression capabilities of FEniCS and its underlying form compiler.  
We add spatially varying quantities using `pdd.spatial`, which has the `add_rule` method.

In order to define in which portions of the simulation domain a rule should apply, Simudo has a powerful **topology engine**. This engine is described in the Simudo reference. It allows regions to be defined without specific reference to the mesh at the time of declaration. 

In [ ]:
from simudo.mesh import CellRegions, FacetRegions
R = CellRegions()
F = FacetRegions()

`R` and `F` are objects that will contain region declarations and can determine the intersections, unions, and boundaries of these regions. Some regions are defined by default, such as `R.domain`, which is the entire simulation domain. 

The temperature is considered to be a spatially varying quantity, though in this problem we will set it to be uniform over the full domain. 

In [ ]:
spatial = pdd.spatial
spatial.add_rule('temperature', R.domain, U("300 K"))

We also define the doping level in each region. Assuming fully ionized dopants, we define the doping level by putting in rules for the static background charge of the ionized impurities. So n-type dopants are given by positive static charge and p-type dopants are given by negative static charge

In [ ]:
spatial.add_rule('poisson/static_rho', R.p, -N_A) #N_A and N_D defined with units
spatial.add_rule('poisson/static_rho', R.n, +N_D)

The regions `R.p` and `R.n` refer to the `name`s that we gave to the `layers` in the `ConstructionHelperLayeredStructure`. `add_rule` allows the rule names (e.g., `poisson/static_rho` or `temperature`) to be anything that you like. Processes request specific property names from `spatial`, as needed.  

### Define material
Frequently, regions of a simulation correspond to different material types, which have their own properties. Materials are generally defined in their own custom files, which can be as complicated as desired. Band edge energies can, for example, depend on local carrier concentrations. In Simudo, these material files create a class that inherits from `Material` and defines a `get_dict` method, returning a dictionary of rules that should be added to `spatial`. 

 Here we create a simple material with only the bare minimum of properties required for this example.

In [ ]:
from simudo.physics import Material

class SimpleMaterial(Material):
    '''Very simple silicon-inspired material.'''

    #name is used for referring to the material
    #it should be different for each material in the problem
    name = 'SimpleMaterial' 

    def get_dict(self):
        d = super().get_dict()
        U = self.unit_registry

        d.update({
            # band parameters, taken from silicon at room temperature
            # source: http://www.ioffe.ru/SVA/NSM/Semicond/Si/bandstr.html
            'CB/energy_level': U('1.12 eV'),
            'VB/energy_level': U('0 eV'),
            'CB/effective_density_of_states': U('3.2e19 1/cm^3'),
            'VB/effective_density_of_states': U('1.8e19 1/cm^3'),

            # electrical properties
            # source: http://www.ioffe.ru/SVA/NSM/Semicond/Si/electric.html
            'CB/mobility': U('1400 cm^2/V/s'),
            'VB/mobility': U(' 450 cm^2/V/s'),

            # poisson
            # http://www.ioffe.ru/SVA/NSM/Semicond/Si/optic.html
            'poisson/permittivity': U('11.7 vacuum_permittivity'),
        })

        return d

The rules that begin with `CB/` or `VB/` give properties associated to the bands named `CB` and `VB`. If we had instead instantiated the conduction band as  
`CB = pdd.easy_add_band(name='C_B',sign=-1)`  
then the `CB/mobility` and other properties would *not* be associated with this band. We would need the material file to have rules `C_B/mobility`, etc.

We now `register` these properties. Since both our `p` and `n` region have the same material, these properties will be registered throughout the domain.

In [ ]:
SimpleMaterial(problem_data=problem_ntrl).register()

As an aside, we can look in the `spatial` object to see the rules that have been defined, which correspond to all of the properties we have so far given.

In [ ]:
print(spatial.value_rules.keys())

### Set up topology (and boundary conditions)
We need to apply boundary conditions to the appropriate external surfaces. In order to do so, we use Simudo's region/facet topology capabilities. We have already met `R.domain`, which refers to the whole simulation domain. Similarly, `R.exterior` is the area outside of the simulation domain. It is useful for defining the external facets of the simulation region. Similarly, `R.exterior_left` is the area to the left of the simulation region, and there is also `R.exterior_right`, `R.exterior_top`, and `R.exterior_bottom`. Further facets can be defined by the user, as we will now do.

We will now define two contacts, which will have Ohmic boundary conditions. We will call them `p_contact` and `n_contact`.

In [ ]:
F.p_contact = R.exterior_left.boundary(R.domain)
F.n_contact = R.domain.boundary(R.exterior_right)

In these lines, we have used the `CellRegions` `boundary` method to define a facet with positive orientation from  `R.exterior_left` toward `R.domain`. In order to keep a consistent sense of positive flow (for current densities), we define the `n_contact` as being the boundary from `R.domain` to `R.exterior_right`.  
We can take the union of the two contact facets, to make boundary conditions easier to apply.

In [ ]:
F.contacts = F.p_contact | F.n_contact

The top and bottom of the simulation domain in this problem are nonconductive boundaries.

In [ ]:
F.exterior = R.exterior.boundary(R.domain)
F.nonconductive = F.exterior - F.contacts.both()

You might think that `F.nonconductive = F.exterior - F.contacts` would have been sufficient to define the nonconductive boundary, but facets are defined by both their location *and* their direction. Here, `F.exterior` is oriented from the outside into the domain, which agrees with the orientation of `F.p_contact` but does not agree with the orientation of `F.n_contact`.  
So `F.nonconductive = F.exterior - F.contacts` would produce a boundary along the top, right, and bottom edges of the domain. The `both` method takes both orientations along a facet. It is particlarly useful when making a new boundary by subtraction.

With these regions defined, we can apply our boundary condition that the perpendicular component of $\vec{E}$ should be zero along the non-conductive edges of the domain.

In [ ]:
mu = pdd.mesh_util #mesh_util has a large number of useful methods for spatial/mesh functions
#mu.zerovec is the DOLFIN zero vector, which we use here to apply boundary conditions
spatial.add_BC('poisson/E', F.nonconductive,U('V/m') * mu.zerovec)

### Local charge neutrality, convergence, and tolerances
We are now (finally) ready to solve the local charge neutrality problem!

In [ ]:
problem_ntrl.pdd.easy_auto_pre_solve()

The first time this (or any) FEM code is run, all of the forms are compiled, producing free form compiler (FFC) notifications. Rerunning a similar problem will not produce these compilation steps. 

Not including compilation time, Simudo solves this problem in a couple of seconds. The output is logged to the console and also to the output files `_info.log` and `_debug.log`, where `_` is the `filename_prefix` in the `TypicalLoggingSetup`, above.
This output contains the time stamp, the stage of calculation (`newton.ntrl` for the charge neutrality solver, in this case), and information on the Newton iterations.  
FEM reduces PDE's to algebra problems. In the case of nonlinear PDE's (such as we have in the device model), we begin with an initial guess trial function $u_0$ and linearize the problem around that guess. We find a $du$ and update the trial function  to $u + du$. With the new approximate solution, we relinearize the problem and iterate until convergence.  
The three columns of output can be used to see the process of convergence. At each step, we are solving for $du$ in a problem of the form  
$A du = b$.  
The $|b|$ output shows the norm of the $b$ vector. The second output shows the largest magnitude value of $du/u$. We do not use that to determine convergence, because it is ill-behaved when any of the components of $u$ is zero. The final output (the combined norm) is the only one used for convergence; the solution is converged when $|du|/(r|u|+a)<1$. The solver has an `extra_iterations` parameter that makes it take a few more Newton steps after reaching convergence, to nail down the solution.  
The combined norm includes the relative tolerance $r$ and the absolute tolerance $a$. The absolute tolerance varies for each type of test function, and there is an overall scaling factor `absolute_tolerance` that scales the $a$ values for all test functions.  
The defaults can all be changed, but the local charge neutrality solver has $r=1e-14$, which you can see in `physics/poisson_drift_diffusion.py:LocalChargeNeutralitySolver`. The `ThermalEquilibriumSolver` has $r=1e-5$ by default. The absolute tolerances for each quantity are (as of this writing)  
$\phi$: 1e-6 V  
$\vec{E}$: 1e-4 V/mesh\_unit  
$\Delta w_k$: 1e-6 eV  
$\vec{j}_k$: 1e-10 A/mesh\_unit$^2$
The `LocalChargeNeutrality` and `ThermalEquilibrium` solvers both have `absolute_tolerance=0.1`, so they have a tighter convergence threshold than these values. The combined tolerance is satisfied when each component of the trial function satisfies either the relative tolerance or the absolute tolerance.


There is not much of interest in the solution at this point, and we will discuss how to output the results in a useful format for plotting later in this notebook. But for now, we can see the electrostatic potential $\phi$ by looking at its value at the various mesh nodes. Note that it is mostly constant through the n- and p- regions. 

In [ ]:
phi_cn=problem_ntrl.pdd.poisson.phi
print(phi_cn.vector().get_local()[0:5])
print(phi_cn.vector().get_local()[-5:])

## Stage 2 - Thermal equilibrium
For now, let's continue to thermal equilibrium. We must construct a new `problem` object, which shares many attributes with `problem_ntrl`. In most code, those are generated by a single function with some `if` statements, but here we will make a new `problem_thmq` object directly.

In [ ]:
problem_thmq = ProblemData(goal="thermal equilibrium", 
                   mesh_data=mesh_data,
                  unit_registry=U)
pdd = problem_thmq.pdd
CB = pdd.easy_add_band(name='CB')
VB = pdd.easy_add_band(name='VB')
spatial = pdd.spatial
spatial.add_rule('temperature', R.domain, U("300 K"))
spatial.add_rule('poisson/static_rho', R.p, -N_A) #N_A and N_D defined with units
spatial.add_rule('poisson/static_rho', R.n, +N_D)
SimpleMaterial(problem_data=problem_thmq).register()
mu = pdd.mesh_util 
spatial.add_BC('poisson/E', F.nonconductive,U('V/m') * mu.zerovec)

# Everything above is the same as in Stage 1. 
# Here is our new boundary condition, imposing the charge-neutrality phi at the contacts
spatial.add_BC('poisson/phi', F.contacts, phi_cn)


We initialize using the solution from the local charge-neutrality solver

In [ ]:
problem_thmq.pdd.initialize_from(problem_ntrl.pdd)

and then solve for thermal equilibrium

In [ ]:
problem_thmq.pdd.easy_auto_pre_solve()

## Stage 3: The full problem

For the full problem, we must add in all of the electrooptical processes (generation/recombination and tunneling) that move carriers between bands as well as imposing boundary conditions on the carrier populations, none of which is required at equilibrium.

Solving the full problem is usually done at a range of optical intensities and/or voltages, and we use a stepper to help us step through those, using the solution from the previous step as the initial guess for the next step. 

[In this example, we will not consider optical fields. But if we were to include them, we would:
1. Define the optical fields
1. Before using the `VoltageStepper`, we would use an `OpticalStepper` which scales the optical fields to slowly increase the optical intensity, using each step's solution as the initial guess for the next one. This is often the slowest part of a simulation.
3. Use the `VoltageStepper` to ramp the voltage. In intermediate band problems, the optical generation problem must be re-solved self consistently as the optical absorption depends on carrier concentrations. This iteration to self-consistency occurs within the Newton iterations of the `VoltageStepper` and is controlled by the `selfconsistent_optics` (default:False) attribute of the `VoltageStepper`.]



In [ ]:
problem = ProblemData(goal="full", 
                  mesh_data=mesh_data,
                  unit_registry=U)
pdd = problem.pdd
CB = pdd.easy_add_band(name='CB')
VB = pdd.easy_add_band(name='VB')
spatial = pdd.spatial
spatial.add_rule('temperature', R.domain, U("300 K"))
spatial.add_rule('poisson/static_rho', R.p, -N_A) #N_A and N_D defined with units
spatial.add_rule('poisson/static_rho', R.n, +N_D)
SimpleMaterial(problem_data=problem).register()
mu = pdd.mesh_util 
spatial.add_BC('poisson/E', F.nonconductive,U('V/m') * mu.zerovec)
# Everything above is the same as in Stage 1,2. 

### Electrooptical processes
Now we define our electrooptical processes. For this example, we will define only an SRHRecombination. Simudo currently has built-in methods for optical absorption, radiative recombination, SRH recombination, and nonradiative Shockley-Read (SR) trapping (eg, between CB and IB). More can be added easily, as describe in Section 4.2 of the reference.

An electrooptical process (EOP) generally defines a source band `src_band` and a destination band `dst_band`. The convention in Simudo is to think of these bands for the associated *generation* process. By default, both generation and recombination processes are included for each EOP, so adding in optical absorption automatically includes the associated bulk radiative recombination and adding in SRH recombination automatically includes the associated thermal generation. In this pn-diode, an optical absorption process would have `src_band = VB` and `dst_band = CB`. For the SRH process, this convention seems slightly backward, but we consider the `src_band = VB` and `dst_band = CB` as well.

In [ ]:
from simudo.physics import SRHRecombination
pdd.easy_add_electro_optical_process(SRHRecombination, dst_band=CB, src_band=VB)

The `SRHRecombination` process imposes a local recombination from `dst_band` to `src_band` through a trap at energy $E_t$ of

$ U = \frac{np - n_i^2}{(p + p_1)\tau_n + (n + n_1)\tau_p}$

where $n_1$ and $p_1$ are the concentration of carriers when the Fermi level is $E_t$, and $\tau_k$ are the associated carrier lifetimes.

To evaluate this process, Simudo looks for `spatial` parameters  
`SRH/energy_level` to define $E_t$
`SRH/bb/tau` where `bb` is replaced by the name of the band (`CB` and `VB` in our case). Then `SRH/CB/tau` defines $\tau_n$ and `SRH/VB/tau` defines $\tau_p$ in this example.

These parameters can easily be set in the material file, but since we didn't include them there, we can define (or override their rules) here. We defined them to be uniform through the domain, though we can also define them to depend on carrier concentrations or any other spatial quantity.

In [ ]:
tau_n = 1e-9 * U("s")
tau_p = 1e-6 * U("s")
E_t = 0.553 * U("eV")
spatial.add_rule('SRH/CB/tau', R.domain, tau_n)
spatial.add_rule('SRH/VB/tau', R.domain, tau_p)
spatial.add_rule('SRH/energy_level', R.domain, E_t)

Note that we have added the `SRHRecombination` process before we defined the parameters required to evaluate it. All processes are stored symbolically until they are evaluated at runtime, so the associated parameters can be defined before or after the processes.

### More boundary conditions
For this example, we  keep the nonconductive boundary condition along the top and bottom surfaces. This is an essential boundary condition on the current densities `j` so that their normal components through those surfaces are zero.

In [ ]:
spatial.add_BC('CB/j', F.nonconductive, U('A/cm^2') * mu.zerovec)
spatial.add_BC('VB/j', F.nonconductive, U('A/cm^2') * mu.zerovec)

At the contacts, we impose ideal boundary conditions: Ohmic condition ($\Delta u = 0$) for majority carriers and zero surface-recombination velocity (SRV) condition ($\vec{j}_k \cdot \hat{n}=0$) where $\hat{n}$ is the surface normal and $\vec{j}_k$ is current density in band $k$. 

In [ ]:
# majority contact
spatial.add_BC('VB/u', F.p_contact, VB.thermal_equilibrium_u)
spatial.add_BC('CB/u', F.n_contact, CB.thermal_equilibrium_u)

# minority contact
spatial.add_BC('CB/j', F.p_contact, U('A/cm^2') * mu.zerovec)
spatial.add_BC('VB/j', F.n_contact, U('A/cm^2') * mu.zerovec)

Finally, we impose the boundary conditions on $\phi$, allowing the external bias $V_{ext}$ to be applied. That parameter will be controlled by the `VoltageStepper`

In [ ]:
import dolfin
V_ext = U.V * dolfin.Constant(0.0) 

`dolfin.Constant` objects tell the compiler that its value doesn't change the properties of any associated form, so they do not need to be recompiled even when the value changes. 

In [ ]:
phi0 = pdd.poisson.thermal_equilibrium_phi 
spatial.add_BC('poisson/phi', F.p_contact, phi0)
spatial.add_BC('poisson/phi', F.n_contact, phi0 - V_ext)

In this case, we are imposing $V_{ext}$ entirely at the n-contact, but that could be easily changed, for example by
```python
spatial.add_BC('poisson/phi', F.p_contact, phi0 + V_ext/2)
spatial.add_BC('poisson/phi', F.n_contact, phi0 - V_ext/2)
```

### Set up stepper and output
We now initialize the full problem from the thermal equilibrium problem and then set up the stepper

In [ ]:
problem.pdd.initialize_from(problem_thmq.pdd)

Choose the values of $V_{ext}$ where we would like output data

In [ ]:
V_targets = np.linspace(0, 1, 11) #These do not get units, which is specified separately

Now we can set up the `VoltageStepper`. It takes in the `problem` object and varies `constants` to meet the `parameter_target_values`. 

In [ ]:
from simudo.physics import VoltageStepper

stepper = VoltageStepper(
        solution=problem, 
        constants=[V_ext],  #specify what constant is changed as we step
        parameter_target_values=V_targets,
        parameter_unit=U.V,
        selfconsistent_optics=False, #We are not considering optics yet
)

We can run this `VoltageStepper` with `stepper.do_loop()`. If we do so right now, it will solve the problem but will not give any output. We need to specify an `OutputWriter` instance, which `VoltageStepper` will call each time one of the `parameter_target_values` is reached. It produces output files, generally with the filename labeling the corresponding parameter_value. 

There are many possible outputs that a user may want. The most common choice is to specify a set of extractors `MetaExtractorIntegrals`, `MetaExtractorBandInfo`

In [ ]:
from simudo.io.output_writer import OutputWriter, MetaExtractorIntegrals, MetaExtractorBandInfo
output_writer = OutputWriter(filename_prefix="out/pn", parameter_name="V",)

In [ ]:
from functools import partial
extractors = (MetaExtractorBandInfo, 
              partial(MetaExtractorIntegrals,
                      facets=F[{'p_contact', 'n_contact'}],
                      cells =R[{'p','n'}]
                     )
             )

`MetaExtractorBandInfo` takes every `band` in the problem and extracts all of its items, including the quasi-Fermi-level and current density.  
`MetaExtractorIntegrals` calculates 
1. For each `facet`, the average (mA/cm$^2$) and total (mA) current for each band through that facet
2. For each `cell`, 
  1. average (1/cm$^3$/s and total (1/s) generation for each band in that cell
  2. total (1/s) generation associated with each EOP for each band in that cell
  3. total (1/s) absorption ($\alpha \Phi$) associated with each optical field for each band in that cell
  
Here, we will extract information at the p- and n-contacts and in the `p` and `n` regions (defined in `layers`). 
  
In addition to these examples, one could output at other `cells`, including `R.domain`, or facets including the pn-junction, `F.junction = R.p.boundary(R.n)`. It's quite flexible.

These outputs go into yaml files with the suffix `plot_meta.yaml`

In [ ]:
output_writer.meta_extractors=extractors

We can also extract line cuts or full 2D information on all of these extracted quantities.  
`plot_1d = True` produces .csv.0 files containing spatial information on a line cut through the domain at `y` halfway through the device.
`plot_mesh= True` produces .xdmf files, suitable for viewing in Paraview, containing all spatial quantities 
`plot_iv = True` produces a separate .csv file containing all of the meta_extractor integrals, with a row for each V, which makes plotting J(V) curves easier.

In [ ]:
output_writer.plot_1d=True
output_writer.line_cut_resolution=1000 #number of points for the 1d sections. Default is 5000
output_writer.plot_mesh=False
output_writer.plot_iv=True

Now we give this `OutputWriter` instance to the `stepper`

In [ ]:
stepper.output_writer=output_writer

Finally, we are ready to solve the pn diode.

In [ ]:
stepper.do_loop()

The output here looks similar to the previous solvers. We can see the main `newton` iterations moving to convergence. `adaptive solve_parameter` in our case is `V_ext`. The `step_size` is the Newton step size. It will grow with successful Newton steps and shrink if they fail to converge, but it will make sure to reach the `V_targets`, which in this case limits its size. Runtime is about a minute, and there are no Newton failures in this example. 

## Simple plotting 


In [ ]:
import pandas as pd

df = pd.read_csv("out/pn_V.csv",skiprows=[1]) #skip the row containing units

In [ ]:
df

In [ ]:
df['V'] = df['sweep_parameter:V']
df['J'] = df['avg:current_CB:p_contact'] + df['avg:current_VB:p_contact']
# df = df[df['V'] != 0]

fig, ax = plt.subplots()
ax.plot(df['V'], df['J'], label="$J(V)$", marker='o')
ax.set_yscale('log')
ax.set_xlabel(r'applied potential ($\mathrm{V}$)')
ax.set_ylabel(r'total current ($\mathrm{mA}/\mathrm{cm}^2$)')
ax.legend()
fig.tight_layout()
# fig.savefig("out/tutorial-pn-diode-JV.png")

We can also extract spatial quantities, for example to make a band diagram.

In [ ]:
plot_V = "0.2"
df1=pd.read_csv(f"out/pn_V={plot_V}.csv.0")

In [ ]:
df1

In [ ]:
fig, axs = plt.subplots(3,1,sharex=True,figsize=(6,10))
ax=axs[0]
ax.plot(df1['coord_x'], df1['Ephi_CB'], color='k')
ax.plot(df1['coord_x'], df1['Ephi_VB'], color='k')
ax.plot(df1['coord_x'], df1['qfl_CB'], color='b', linestyle='--', label=r'$E_{F,C}$')
ax.plot(df1['coord_x'], df1['qfl_VB'], color='r', linestyle='--', label=r'$E_{F,V}$')
ax.set_title(f"V = {plot_V} V")
# ax.set_xlabel(r'x ($\mu$m)')
ax.set_ylabel(r'Energy (eV)')
ax.legend()

ax=axs[1]
ax.plot(df1['coord_x'], df1['u_CB'], color='b',label=r'n')
ax.plot(df1['coord_x'], df1['u_VB'], color='r',label=r'p')
ax.set_ylabel(r'carrier concentration (1/cm$^3$)')
ax.set_yscale('log')
ax.legend()

ax=axs[2]
ax.plot(df1['coord_x'], df1['g_SRH_CB'], color='k',label=r'g_{SRH}')
ax.set_xlabel(r'x ($\mu$m)')
ax.set_ylabel(r'SRH generation rate (1/cm$^3$/s)')
ax.set_yscale('symlog')

While J(V) is well resolved, in the depletion region there are locations where the carrier concentration is less well resolved, though some of that error comes from projecting the solutions on the calculation space to the output space. A tighter mesh (eg, setting `factor=1.05` in `mesh`) can reduce the scale of these deviations, if desired, but their overall magnitude is not significant.